In [1]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn import metrics
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score
import spacy
from spacy.lang.es.examples import sentences 
import es_core_news_sm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

## Carga de datos

In [2]:
trn = pd.read_csv("S3_P3.csv")
trn = trn.drop(columns=['Unnamed: 0'])
tst = pd.read_csv("test.csv")

In [3]:
tst.head(10)

,comment_id,sentence_pos,reply_to,sentence
0,0,1,0,"A ver si lo he entendido. Un avión marroquí, l..."
1,0,2,0,Nadie ha controlado a los pasajeros y estos de...
2,0,3,0,"Definitivamente, en este pais no cabe un tonto..."
3,0,4,0,"Da vergüenza ser español, nos toman por imbéci..."
4,0,5,0,"No hay ningún país normal en el que pase esto,..."
5,0,6,0,"Aquí no, porque en España los delincuentes, lo..."
6,1,1,1,Uy.....Por el momento y hasta que no obtengamo...
7,1,2,1,Y lo peor de todo...Nadie habla catalán ...por...
8,2,1,234,"falso, esos aviones evitan siempre sobrevolar ..."
9,3,1,2,Señores tenemos un sosiata ganador.


## Word Embedding para test

In [4]:
nlp = es_core_news_sm.load()

In [5]:
nltk.download('stopwords')
palabrasparadoras = stopwords.words('spanish') # lista de stopwords del español
print(palabrasparadoras)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
tst_sentences=tst["sentence"].to_list()

pals={}
for n,s in enumerate(tst_sentences):
    doc = nlp(s)
    pals[n] = []
    for d in doc:
        if d.text.lower() not in palabrasparadoras and (d.pos_!='PUNCT' and d.pos_!='SPACE'):
            pals[n].append(d.text)
            #print(d.text,  d.pos_, d.lemma_)
            
print(len(pals))
print(pals) # diccionario con oraciones (en forma de listas) con palabras filtradas

1812
{0: ['ver', 'si', 'entendido', 'avión', 'marroquí', 'lleno', 'hijos', 'Mohamed', 'aterrizado', 'urgencia', 'aeropuerto', 'Palma'], 1: ['Nadie', 'controlado', 'pasajeros', 'deambulan', 'bola', 'pistas', 'aeropuerto', 'colado', 'país'], 2: ['Definitivamente', 'pais', 'cabe', 'tonto'], 3: ['Da', 'vergüenza', 'ser', 'español', 'toman', 'imbéciles'], 4: ['ningún', 'país', 'normal', 'pase', 'si', 'haces', 'detienen', 'disparan'], 5: ['Aquí', 'España', 'delincuentes', 'ilegales', 'chusma', 'derechos', 'propios', 'españoles'], 6: ['Uy', 'momento', 'obtengamos', 'información', 'contrastada', 'aquí', 'muchas', 'cosas', 'casan', 'resultan', 'verdaderamente', 'inquietantes'], 7: ['peor', 'Nadie', 'habla', 'catalán', 'parece'], 8: ['falso', 'aviones', 'evitan', 'siempre', 'sobrevolar', 'africa'], 9: ['Señores', 'sosiata', 'ganador'], 10: ['prueba', 'si', 'hiciera', 'falta', 'miseria', 'país'], 11: ['Gobierno', 'conseguido', 'enemigos', 'buscando', 'España', 'signifique', 'mundo'], 12: ['presid

### Word Embedding

In [24]:
import numpy
import scipy
from gensim.models import Word2Vec, KeyedVectors

In [19]:
embedd = Word2Vec(min_count = 1)
embedd.build_vocab(pals.values())
embedd.train(pals.values(), total_examples=embedd.corpus_count, epochs=embedd.epochs)

(69160, 70415)

In [20]:
vectors = {}
for n,l in pals.items():
    vectors[n] = [embedd.wv.get_vector(word) for word in l]

In [21]:
nombresfilas = [pal for l in pals.values() for pal in l]
dfvectors = [v for l in vectors.values() for v in l]
pd.DataFrame(dfvectors, index = nombresfilas).head(10)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
ver,-0.010456,0.011002,-0.006835,-0.002196,-0.004415,-0.007433,-0.007461,-0.004202,-0.005710,-0.002287,...,0.010295,0.007874,0.002163,-0.000996,-0.003495,-0.003026,0.009877,-0.005033,-0.006132,0.010472
si,-0.001390,0.002406,0.006359,0.009883,-0.008791,-0.010824,0.007323,0.014561,-0.006265,-0.005278,...,0.004230,0.001188,0.004408,0.000943,0.013139,0.007182,-0.007310,-0.009126,0.001123,0.006971
entendido,0.009926,-0.005440,0.002447,0.008909,-0.002638,-0.000867,0.001667,0.006277,0.006507,-0.005069,...,0.006590,-0.005519,-0.007946,0.002131,-0.000831,0.000520,0.001875,0.009254,0.008635,0.007195
avión,0.006454,-0.000027,0.008653,-0.009234,-0.007759,-0.008681,-0.003493,0.007933,-0.004406,-0.009560,...,0.009308,0.002252,-0.000818,0.006183,-0.003674,0.001035,0.010585,-0.007117,0.004145,-0.008053
marroquí,0.002768,-0.009301,-0.003935,-0.009127,0.002080,-0.001145,-0.009623,-0.003897,-0.006317,-0.010158,...,0.000176,-0.008352,0.006603,-0.004543,0.006296,-0.008569,-0.003860,0.000629,0.001627,0.002387
lleno,0.006862,-0.003294,0.007599,0.002279,-0.003080,0.007820,-0.001901,-0.001219,0.000028,0.007003,...,0.007706,0.009147,-0.006312,0.001984,-0.001589,0.008955,-0.008364,0.005098,-0.006957,-0.007605
hijos,-0.009377,-0.009215,-0.002282,0.001375,0.001584,0.003342,0.004514,0.009214,-0.005920,-0.005323,...,0.003649,-0.000902,0.001547,0.007676,0.004591,0.003073,0.010199,-0.007294,0.006092,0.009907
Mohamed,-0.004066,0.006098,-0.009308,-0.008995,0.005936,0.000905,0.003548,0.005014,0.004871,-0.006447,...,0.002216,0.000755,0.002615,0.000388,-0.006152,0.005830,0.001600,-0.007025,0.001514,-0.002070
aterrizado,-0.000442,0.008686,0.009161,0.002300,-0.007296,0.008652,-0.004231,0.005171,-0.004692,-0.008734,...,-0.006922,-0.007487,0.000836,0.001179,0.000950,-0.009443,-0.004177,-0.003917,-0.006998,0.005013
urgencia,0.008823,0.006886,-0.000918,0.002321,0.003458,-0.000238,-0.002278,0.007794,0.003209,-0.007091,...,-0.003858,0.002376,0.005153,0.006923,-0.000840,-0.006747,0.000146,-0.009309,-0.000076,-0.003177


In [25]:
meanvectors={}

for oracion in vectors:
    try:
        meanvectors[oracion]=(sum(vector for vector in vectors[oracion]))/len(vectors[oracion])
        
    except:
        meanvectors[oracion]=numpy.zeros(100)

In [27]:
tst = pd.DataFrame(meanvectors.values()) # los nombres de las filas son la posición de las oraciones
tst.head(10)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.002213,0.000321,0.000988,0.000962,-0.001649,-0.000305,-0.000937,0.003046,-0.000908,-0.003438,...,0.002876,0.000397,0.000087,0.002232,0.000784,0.000371,0.000774,-0.004465,0.000690,0.001843
1,0.001865,0.001121,-0.002208,-0.003140,-0.000903,0.001515,-0.000349,0.000452,-0.003069,-0.001080,...,0.001816,0.000003,0.001091,-0.000643,-0.000472,0.004095,0.000813,-0.001137,0.004791,0.001646
2,0.002517,0.000407,-0.000981,-0.000749,-0.003680,-0.002441,0.002913,0.001173,-0.000912,-0.001703,...,0.002552,-0.001678,0.003059,-0.000422,0.001396,-0.001447,-0.001657,-0.001723,-0.000844,0.000354
3,0.001186,0.003253,-0.002899,0.000129,0.002872,-0.004151,0.003876,0.005238,-0.000318,-0.005208,...,0.000070,0.000257,0.000157,0.001344,-0.000078,-0.001233,-0.002709,-0.004168,-0.000682,0.000389
4,-0.000354,0.000880,-0.000214,-0.000296,-0.002124,0.000001,0.000924,0.002087,-0.000909,0.000109,...,0.003008,0.003539,0.003418,-0.004429,0.005080,0.003129,-0.000225,-0.000331,0.000584,-0.001135
5,0.001053,0.001829,0.002351,-0.001789,0.004352,-0.000756,0.000249,0.003694,-0.000800,-0.002301,...,0.001530,0.000019,0.003061,0.000966,0.002289,-0.000310,-0.003826,0.002001,-0.002353,-0.003380
6,-0.000020,0.000491,-0.000957,-0.001586,-0.001782,-0.001803,-0.001052,-0.002248,0.000030,0.001267,...,0.000047,0.002833,-0.001929,0.000617,0.001301,0.004476,0.003513,-0.000475,0.000743,0.002024
7,-0.002768,-0.004781,0.002229,-0.000190,0.003610,-0.005081,0.000842,0.001869,-0.003887,-0.000306,...,0.003218,-0.002475,-0.008743,0.002025,-0.003303,0.004382,-0.004017,0.000830,-0.002670,0.003212
8,0.004221,-0.002742,0.000131,0.003081,-0.000379,-0.004164,0.001769,0.001430,-0.002208,0.000231,...,-0.003964,-0.001037,0.003744,-0.001305,-0.004887,0.002195,-0.002047,0.001312,-0.000277,0.002607
9,0.000649,0.000774,-0.001767,0.000536,0.003636,-0.005752,-0.001330,0.005114,0.000654,-0.000997,...,0.003587,-0.002282,0.002121,-0.000548,-0.001893,0.002353,0.002020,0.002653,-0.000571,-0.001397


## Implementamos los modelos

Mezclamos los datos de entrenamiento:

In [28]:
trn.head(10)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,stereotype
0,0.001318,0.003393,0.002340,-0.002296,0.004027,0.002442,0.000703,0.001406,-0.002482,0.000745,...,-0.001195,-0.000311,-0.000729,0.001500,-0.000136,-0.002057,0.004157,0.002924,0.003697,0
1,0.001163,0.002280,0.005396,-0.005706,0.003198,0.001721,0.002030,0.004187,-0.003248,0.007304,...,-0.002858,0.007483,-0.001480,0.005370,-0.002683,0.005325,-0.001532,0.000768,-0.002241,0
2,-0.001684,0.003351,-0.000300,0.000164,0.000150,-0.003695,0.001025,0.005774,0.001311,-0.000973,...,0.000300,0.000582,-0.004117,0.001826,0.002370,0.000677,-0.001318,-0.000238,0.002707,0
3,0.001581,-0.000432,0.002926,-0.001243,0.000831,-0.002557,0.003618,0.003611,0.000520,-0.000479,...,0.001939,-0.002411,-0.002368,0.001319,-0.000663,0.004117,-0.000327,0.003946,0.004706,0
4,-0.000496,-0.000016,0.007480,-0.002841,-0.001257,-0.001341,0.001828,0.003550,0.003153,0.001166,...,0.000466,0.005146,0.002355,0.002462,0.003463,0.002679,0.001108,-0.001922,0.000507,0
5,0.000717,0.001005,-0.000263,0.000679,0.001691,-0.003206,-0.002786,0.006954,-0.001253,0.001588,...,0.000191,-0.002558,-0.002527,0.007731,0.003119,-0.003920,0.001947,-0.004730,-0.002866,0
6,-0.004574,-0.001306,0.000020,-0.002135,-0.000803,0.000261,-0.000666,0.002697,0.000333,0.002623,...,-0.002448,0.001542,0.000938,0.005675,-0.000728,0.002071,-0.001132,0.002149,-0.001617,0
7,-0.007377,0.000183,0.004989,-0.006015,0.002102,-0.002705,-0.006664,0.009896,-0.009904,0.007932,...,0.008981,0.001597,0.000005,-0.006048,0.009836,0.006450,-0.005930,0.005494,0.005669,0
8,-0.000654,0.001425,0.000660,0.001815,-0.002990,-0.001975,0.000159,0.004599,-0.004364,0.000639,...,-0.003120,0.004801,-0.000113,0.003341,0.002509,0.002654,0.000550,0.001538,0.003378,0
9,0.004029,0.002029,0.004909,-0.000626,0.000693,-0.003865,0.001327,-0.008393,-0.000097,-0.005665,...,0.001387,0.005404,-0.005205,0.001415,0.006878,0.000329,-0.003058,-0.003888,0.002836,0


In [34]:
X = trn.drop("stereotype", axis=1).values
y = trn["stereotype"].values

In [35]:
X_train, y_train = shuffle(X, y)

Y los datos de test a predecir:

In [36]:
X_test = tst.values

### Stacking

In [110]:
import time

tini = time.process_time()
estimators=[('mlp',MLPClassifier(random_state=1, max_iter=300)),('svr', svm.SVC(kernel='rbf', C=10))]
clf=StackingClassifier(estimators=estimators,final_estimator=RandomForestClassifier())
clf.fit(X_train, y_train)
y_test_1 = clf.predict(X_test)

print(y_test_1)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\guill\Anaconda3\lib\site-packages\sklearn\n

[1 0 0 ... 0 0 0]

Ha tardado 165.33 segundos


In [111]:
tst_1 = pd.read_csv("test.csv")
attempt_1 = pd.DataFrame(y_test_1)
attempt_1 = attempt_1.rename(columns={0:'stereotype'})
tst_1 = tst_1.join(attempt_1)

In [112]:
tst_1.head(10)

,comment_id,sentence_pos,reply_to,sentence,stereotype
0,0,1,0,"A ver si lo he entendido. Un avión marroquí, l...",1
1,0,2,0,Nadie ha controlado a los pasajeros y estos de...,0
2,0,3,0,"Definitivamente, en este pais no cabe un tonto...",0
3,0,4,0,"Da vergüenza ser español, nos toman por imbéci...",0
4,0,5,0,"No hay ningún país normal en el que pase esto,...",0
5,0,6,0,"Aquí no, porque en España los delincuentes, lo...",0
6,1,1,1,Uy.....Por el momento y hasta que no obtengamo...,1
7,1,2,1,Y lo peor de todo...Nadie habla catalán ...por...,0
8,2,1,234,"falso, esos aviones evitan siempre sobrevolar ...",0
9,3,1,2,Señores tenemos un sosiata ganador.,0


In [113]:
print("Conteo de clases de stereotype: ")
tst_1["stereotype"].value_counts()

Conteo de clases de stereotype: 


0    1739
1      73
Name: stereotype, dtype: int64

In [115]:
73/1812

0.04028697571743929

### Bagging SVM

In [61]:
import time

tini = time.process_time()

clf=BaggingClassifier(base_estimator=svm.SVC(kernel='rbf', C=10),n_estimators=10)
clf.fit(X_train, y_train)
y_test_2 = clf.predict(X_test)

print(y_test_2)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

[1 0 0 ... 0 1 0]

Ha tardado 16.62 segundos


In [62]:
tst_2 = pd.read_csv("test.csv")
attempt_2 = pd.DataFrame(y_test_2)
attempt_2 = attempt_2.rename(columns={0:'stereotype'})
tst_2 = tst_2.join(attempt_2)

In [63]:
tst_2.head(10)

,comment_id,sentence_pos,reply_to,sentence,stereotype
0,0,1,0,"A ver si lo he entendido. Un avión marroquí, l...",1
1,0,2,0,Nadie ha controlado a los pasajeros y estos de...,0
2,0,3,0,"Definitivamente, en este pais no cabe un tonto...",0
3,0,4,0,"Da vergüenza ser español, nos toman por imbéci...",0
4,0,5,0,"No hay ningún país normal en el que pase esto,...",0
5,0,6,0,"Aquí no, porque en España los delincuentes, lo...",0
6,1,1,1,Uy.....Por el momento y hasta que no obtengamo...,1
7,1,2,1,Y lo peor de todo...Nadie habla catalán ...por...,0
8,2,1,234,"falso, esos aviones evitan siempre sobrevolar ...",0
9,3,1,2,Señores tenemos un sosiata ganador.,0


In [64]:
print("Conteo de clases de stereotype: ")
tst_2["stereotype"].value_counts()

Conteo de clases de stereotype: 


0    1576
1     236
Name: stereotype, dtype: int64

In [66]:
236/1812

0.13024282560706402

### AdaBoost Random Forest 

In [67]:
import time

tini = time.process_time()

clf=AdaBoostClassifier(base_estimator=RandomForestClassifier(),n_estimators=10)
clf.fit(X_train, y_train)
y_test_3 = clf.predict(X_test)

print(y_test_3)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

[1 0 0 ... 0 1 0]

Ha tardado 68.34 segundos


In [68]:
tst_3 = pd.read_csv("test.csv")
attempt_3 = pd.DataFrame(y_test_3)
attempt_3 = attempt_3.rename(columns={0:'stereotype'})
tst_3 = tst_3.join(attempt_3)

In [69]:
tst_3.head(10)

,comment_id,sentence_pos,reply_to,sentence,stereotype
0,0,1,0,"A ver si lo he entendido. Un avión marroquí, l...",1
1,0,2,0,Nadie ha controlado a los pasajeros y estos de...,0
2,0,3,0,"Definitivamente, en este pais no cabe un tonto...",0
3,0,4,0,"Da vergüenza ser español, nos toman por imbéci...",0
4,0,5,0,"No hay ningún país normal en el que pase esto,...",0
5,0,6,0,"Aquí no, porque en España los delincuentes, lo...",0
6,1,1,1,Uy.....Por el momento y hasta que no obtengamo...,0
7,1,2,1,Y lo peor de todo...Nadie habla catalán ...por...,0
8,2,1,234,"falso, esos aviones evitan siempre sobrevolar ...",0
9,3,1,2,Señores tenemos un sosiata ganador.,0


In [70]:
print("Conteo de clases de stereotype: ")
tst_3["stereotype"].value_counts()

Conteo de clases de stereotype: 


0    1648
1     164
Name: stereotype, dtype: int64

In [71]:
164/1812

0.09050772626931568

### SVM óptimo

In [72]:
import time

tini = time.process_time()

clf=svm.SVC(kernel='rbf', C=10, random_state=42)
clf.fit(X_train, y_train)
y_test_4 = clf.predict(X_test)

print(y_test_4)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

[1 0 1 ... 0 1 0]

Ha tardado 3.78 segundos


In [73]:
tst_4 = pd.read_csv("test.csv")
attempt_4 = pd.DataFrame(y_test_4)
attempt_4 = attempt_4.rename(columns={0:'stereotype'})
tst_4 = tst_4.join(attempt_3)

In [74]:
tst_4.head(10)

,comment_id,sentence_pos,reply_to,sentence,stereotype
0,0,1,0,"A ver si lo he entendido. Un avión marroquí, l...",1
1,0,2,0,Nadie ha controlado a los pasajeros y estos de...,0
2,0,3,0,"Definitivamente, en este pais no cabe un tonto...",1
3,0,4,0,"Da vergüenza ser español, nos toman por imbéci...",0
4,0,5,0,"No hay ningún país normal en el que pase esto,...",0
5,0,6,0,"Aquí no, porque en España los delincuentes, lo...",0
6,1,1,1,Uy.....Por el momento y hasta que no obtengamo...,1
7,1,2,1,Y lo peor de todo...Nadie habla catalán ...por...,0
8,2,1,234,"falso, esos aviones evitan siempre sobrevolar ...",0
9,3,1,2,Señores tenemos un sosiata ganador.,0


In [75]:
print("Conteo de clases de stereotype: ")
tst_4["stereotype"].value_counts()

Conteo de clases de stereotype: 


0    1593
1     219
Name: stereotype, dtype: int64

In [76]:
219/1812

0.12086092715231789

### Red neuronal

In [100]:
import time
from sklearn.neural_network import MLPClassifier

tini = time.process_time()

clf = MLPClassifier(random_state=9, max_iter=700, solver = 'lbfgs')
clf.fit(X_train, y_train)
y_test_5 = clf.predict(X_test)

print(y_test_5)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

[1 1 1 ... 0 1 0]

Ha tardado 97.22 segundos


C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [101]:
tst_5 = pd.read_csv("test.csv")
attempt_5 = pd.DataFrame(y_test_5)
attempt_5 = attempt_5.rename(columns={0:'stereotype'})
tst_5 = tst_5.join(attempt_3)

In [102]:
tst_5.head(10)

,comment_id,sentence_pos,reply_to,sentence,stereotype
0,0,1,0,"A ver si lo he entendido. Un avión marroquí, l...",1
1,0,2,0,Nadie ha controlado a los pasajeros y estos de...,0
2,0,3,0,"Definitivamente, en este pais no cabe un tonto...",0
3,0,4,0,"Da vergüenza ser español, nos toman por imbéci...",0
4,0,5,0,"No hay ningún país normal en el que pase esto,...",0
5,0,6,0,"Aquí no, porque en España los delincuentes, lo...",0
6,1,1,1,Uy.....Por el momento y hasta que no obtengamo...,0
7,1,2,1,Y lo peor de todo...Nadie habla catalán ...por...,0
8,2,1,234,"falso, esos aviones evitan siempre sobrevolar ...",0
9,3,1,2,Señores tenemos un sosiata ganador.,0


In [103]:
print("Conteo de clases de stereotype: ")
tst_5["stereotype"].value_counts()

Conteo de clases de stereotype: 


0    1648
1     164
Name: stereotype, dtype: int64

In [104]:
164/1812

0.09050772626931568

### Bagging Árboles de decisión

In [81]:
import time

tini = time.process_time()

clf=BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion = 'gini', max_depth=30),n_estimators=10)
clf.fit(X_train, y_train)
y_test_6 = clf.predict(X_test)

print(y_test_6)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

[0 0 0 ... 0 0 0]

Ha tardado 5.22 segundos


In [82]:
tst_6 = pd.read_csv("test.csv")
attempt_6 = pd.DataFrame(y_test_6)
attempt_6 = attempt_6.rename(columns={0:'stereotype'})
tst_6 = tst_6.join(attempt_6)

In [83]:
tst_6.head(10)

,comment_id,sentence_pos,reply_to,sentence,stereotype
0,0,1,0,"A ver si lo he entendido. Un avión marroquí, l...",0
1,0,2,0,Nadie ha controlado a los pasajeros y estos de...,0
2,0,3,0,"Definitivamente, en este pais no cabe un tonto...",0
3,0,4,0,"Da vergüenza ser español, nos toman por imbéci...",0
4,0,5,0,"No hay ningún país normal en el que pase esto,...",0
5,0,6,0,"Aquí no, porque en España los delincuentes, lo...",0
6,1,1,1,Uy.....Por el momento y hasta que no obtengamo...,0
7,1,2,1,Y lo peor de todo...Nadie habla catalán ...por...,0
8,2,1,234,"falso, esos aviones evitan siempre sobrevolar ...",0
9,3,1,2,Señores tenemos un sosiata ganador.,0


In [84]:
print("Conteo de clases de stereotype: ")
tst_6["stereotype"].value_counts()

Conteo de clases de stereotype: 


0    1655
1     157
Name: stereotype, dtype: int64

In [105]:
157/1812

0.08664459161147903

### Regresión Logística

In [106]:
import time

tini = time.process_time()

#X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                          test_size=0.25, shuffle = True, random_state=42)

clf = BaggingClassifier(base_estimator=LogisticRegression(C = 0.7839693033701929, penalty = 'l2'),n_estimators=10)
clf.fit(X_train, y_train)
y_test_7 = clf.predict(X_test)

print(y_test_7)


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

[0 0 0 ... 0 0 0]

Ha tardado 1.22 segundos


In [107]:
tst_6 = pd.read_csv("test.csv")
attempt_6 = pd.DataFrame(y_test_6)
attempt_6 = attempt_6.rename(columns={0:'stereotype'})
tst_6 = tst_6.join(attempt_6)

In [108]:
tst_6.head(10)

,comment_id,sentence_pos,reply_to,sentence,stereotype
0,0,1,0,"A ver si lo he entendido. Un avión marroquí, l...",0
1,0,2,0,Nadie ha controlado a los pasajeros y estos de...,0
2,0,3,0,"Definitivamente, en este pais no cabe un tonto...",0
3,0,4,0,"Da vergüenza ser español, nos toman por imbéci...",0
4,0,5,0,"No hay ningún país normal en el que pase esto,...",0
5,0,6,0,"Aquí no, porque en España los delincuentes, lo...",0
6,1,1,1,Uy.....Por el momento y hasta que no obtengamo...,0
7,1,2,1,Y lo peor de todo...Nadie habla catalán ...por...,0
8,2,1,234,"falso, esos aviones evitan siempre sobrevolar ...",0
9,3,1,2,Señores tenemos un sosiata ganador.,0


In [109]:
print("Conteo de clases de stereotype: ")
tst_6["stereotype"].value_counts()

Conteo de clases de stereotype: 


0    1655
1     157
Name: stereotype, dtype: int64

## Evaluación y generación de los csv

In [116]:
import os
    
def save_submission(results_df, team_name, task_number, results_path='./', attempt=0):
    if task_number == 1:
        columns = ['comment_id', 'sentence_pos', 'stereotype']
    elif task_number == 2:
        columns = ['comment_id', 'sentence_pos', 'stereotype', 'xenophobia', 'suffering', 'economic', 'migration',
                   'culture', 'benefits', 'health', 'security', 'dehumanisation', 'others']
    else:
        raise ValueError(f'The task number must be 1 or 2')

    results_df = results_df[columns]
    assert pd.isna(results_df).values.sum() == 0
    results_df.to_csv(os.path.join(results_path, f'{team_name}_task{task_number}_{attempt}.csv'), index=False)

In [117]:
save_submission(results_df=tst_2, team_name='gfermuo',task_number=1,
                attempt=1)

In [118]:
save_submission(results_df=tst_4, team_name='gfermuo',task_number=1,
                attempt=2)

In [119]:
save_submission(results_df=tst_3, team_name='gfermuo',task_number=1,
                attempt=3)

In [120]:
371/1812

0.20474613686534215